In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import re

from src.LLM_compare import compare_using_gemini
from src.call_models import google_connect
from my_access_keys import google_access_key

from tqdm.notebook import tqdm

In [2]:
def parse_query(text):
    pattern = r"<([^>]+)>(.*?)</\1>"
    matches = re.findall(pattern, text, re.DOTALL)
    return {key: value.strip() for key, value in matches}

In [4]:
# file_name = 'compare_csv/arc_ai2_chall_train_top_50.csv'
file_name = 'compare_csv/arc_ai2_chall_train_91_200_for_comparison.csv'
df = pd.read_csv(file_name)
text_df = pd.read_csv(file_name[:-4] + '-text.csv')
display(df.head(3))
display(text_df.head(1))

,original,claude_v2_refine,gemini_pro_think_v2
0,<question>Cellular respiration results in the ...,<question>נשימה תאית מובילה לייצור מולקולות אד...,<question>תוצאתה של נשימה תאית היא ייצור מולקו...
1,<question>The following mathematical expressio...,<question>הביטויים המתמטיים הבאים מייצגים ארבע...,<question>הביטויים המתמטיים הבאים מייצגים ארבע...
2,<question>Data in tables may also be presented...,<question>נתונים בטבלאות יכולים להיות מוצגים ג...,<question>נתונים בטבלאות יכולים להיות מוצגים ג...


,original,claude_v2_refine text,gemini_pro_think_v2 text
0,<question>Cellular respiration results in the ...,First translation attempt:\n\n<question>נשימה ...,**Cellular Respiration Translation Breakdown**...


In [5]:
df.shape

(110, 3)

In [8]:
def color_gradient(val):
    val = val / 50
    # g = int(255 * val ** 2)
    g = int(255 * val ** 0.5)
    r = 255 - g
    return f'background-color: rgb({r}, {g}, 0)'

In [9]:
corr = {}
for c_1 in df.columns:
    for c_2 in df.columns:
        corr[(c_1, c_2)] = (df[c_1] == df[c_2]).sum()

use_keys = list(df.columns)
use_keys = sorted(use_keys)

for sub in ['claude', 'gemini']:
    print(sub)
    use_keys_sub = [k for k in use_keys if sub in k]
    corr_df = pd.DataFrame(index=use_keys_sub, columns=use_keys_sub)
    for k_1 in use_keys_sub:
        for k_2 in use_keys_sub:
            corr_df.loc[k_1, k_2] = corr[(k_1, k_2)]
    styled_df = corr_df.style.format('{:.3f}').map(color_gradient)
    display(styled_df)
    print()

claude


,claude_v2_refine
claude_v2_refine,110.000



gemini


,gemini_pro_think_v2
gemini_pro_think_v2,110.000


In [10]:
from nltk.translate.meteor_score import meteor_score as meteor_score_func
import nltk

nltk.download('wordnet')

def calc_meteor_score(references, candidates):
    scores = []
    for i in range(len(references)):
        scores.append(meteor_score_func(references[i], candidates[i]))

    return np.mean(scores)

[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
calc_meteor_score(
    [[i.split()] for i in df['claude_v2_refine'].values],
    [i.split() for i in df['gemini_pro_think_v2'].values],
)

0.8058360647941835

In [12]:
calc_meteor_score(
    [[i.split()] for i in df['gemini_pro_think_v2'].values],
    [i.split() for i in df['claude_v2_refine'].values],
)

0.8081313755976588

In [13]:
def met_color_gradient(val):
    g = int(255 * val ** 3)
    r = 255 - g
    return f'background-color: rgb({r}, {g}, 0)'


met = {}
for c_1 in tqdm(df.columns):
    for c_2 in df.columns:
        met[(c_1, c_2)] = calc_meteor_score(
            [[i.split()] for i in df[c_1].values],
            [i.split() for i in df[c_2].values],
        )

met_df = pd.DataFrame(index=df.columns, columns=df.columns)
for k_1 in df.columns:
    for k_2 in df.columns:
        met_df.loc[k_1, k_2] = met[(k_1, k_2)]
styled_df = met_df.style.format('{:.3f}').map(met_color_gradient)
display(styled_df)


# for sub in ['claude', 'gemini']:
#     print(sub)
#     use_keys_sub = [k for k in use_keys if sub in k]
#     met_df = pd.DataFrame(index=use_keys_sub, columns=use_keys_sub)
#     for k_1 in use_keys_sub:
#         for k_2 in use_keys_sub:
#             met_df.loc[k_1, k_2] = met[(k_1, k_2)]
#     styled_df = met_df.style.format('{:.3f}').map(met_color_gradient)
#     display(styled_df)
#     print()

  0%|          | 0/3 [00:00<?, ?it/s]

,original,claude_v2_refine,gemini_pro_think_v2
original,1.000,0.180,0.182
claude_v2_refine,0.206,1.000,0.806
gemini_pro_think_v2,0.209,0.808,1.000


# LLM comparison

In [56]:
from src.LLM_compare import CHOOSE_BEST_INSTRUCT

In [57]:
print(CHOOSE_BEST_INSTRUCT)

Your task is choose the best translation between two possible translatoins from English to Hebrew.
Choose the option which have the best fluency, while maintaining the same semantic meaning as the original.

<input format>
English:
{Original text}

First option:
{First Translation option}

Second option:
{Second Translation option}
</input format>


In [46]:
mapping_this = {
    'First option': 1,
    'Second option': 0,
}

# indx = 0
lst = []
for indx in tqdm(range(10)):
    original = df['original'].iloc[indx]
    option_A = df['claude_v1_refine'].iloc[indx]
    option_B = df['gemini_pro_v1_think'].iloc[indx]
    if option_A == option_B:
        lst.append((('', ''), ('', '')))
        continue
    one_side = []
    second_side = []
    for i in range(5):
        one_side.append(compare_using_gemini(google_client, original, option_A, option_B))
        second_side.append(compare_using_gemini(google_client, original, option_B, option_A))
    one_side = [mapping_this[i[0]] for i in one_side]
    second_side = [mapping_this[i[0]] for i in second_side]

    lst.append(((sum(one_side)) / 5, (sum(second_side)) / 5))

    # if output[1] != output_swap[1]:
    #     print(f'{indx:3} - Well well well.....', output[0])
    # else:
    #     print(f'{indx:3} - Got the same result after swapping')
    # lst.append((output, output_swap))

  0%|          | 0/10 [00:00<?, ?it/s]

In [51]:
for i in range(10):
    if i == 5:
        print(i)
        continue
    print(i, lst[i][0], lst[i][1])

0 0.4 0.0
1 1.0 0.4
2 1.0 0.4
3 0.0 1.0
4 0.0 1.0
5
6 0.6 0.6
7 0.2 1.0
8 0.4 0.2
9 1.0 0.0


In [20]:
for i in range(20):
    print(i, lst[i][0][0], lst[i][1][0])

0 First option Second option
1 First option Second option
2 First option First option
3 Second option First option
4 Second option First option
5  
6 Second option First option
7 Second option First option
8 Second option Second option
9 First option Second option
10 Second option First option
11 Second option First option
12 Second option First option
13 First option Second option
14 Second option Second option
15 Second option First option
16 Second option First option
17 First option Second option
18 Second option First option
19 Second option First option


In [ ]:
pd.Series([i[0][0] for i in lst]).value_counts()

In [ ]:
pd.Series([i[1][0] for i in lst]).value_counts()

# To Comparison Files

In [14]:
compare_a = 'claude_v2_refine'
compare_b = 'gemini_pro_think_v2'
new_df = df[['original']].copy()
options = np.array([compare_a, compare_b])
first_is = np.random.choice([0, 1], size=new_df.shape[0], replace=True)
second_is = ~first_is
first_is = options[first_is]
second_is = options[second_is]

new_df['model 1'] = first_is
new_df['model 2'] = second_is

new_df['option 1'] = df.head(new_df.shape[0]).apply(lambda x: x[first_is[x.name]], axis=1)
new_df['option 2'] = df.head(new_df.shape[0]).apply(lambda x: x[second_is[x.name]], axis=1)

new_df.head(3)

,original,model 1,model 2,option 1,option 2
0,<question>Cellular respiration results in the ...,claude_v2_refine,gemini_pro_think_v2,<question>נשימה תאית מובילה לייצור מולקולות אד...,<question>תוצאתה של נשימה תאית היא ייצור מולקו...
1,<question>The following mathematical expressio...,gemini_pro_think_v2,claude_v2_refine,<question>הביטויים המתמטיים הבאים מייצגים ארבע...,<question>הביטויים המתמטיים הבאים מייצגים ארבע...
2,<question>Data in tables may also be presented...,gemini_pro_think_v2,claude_v2_refine,<question>נתונים בטבלאות יכולים להיות מוצגים ג...,<question>נתונים בטבלאות יכולים להיות מוצגים ג...


In [15]:
def right_1(x):
    if x['model 1'] == compare_a:
        return x['option 1']
    else:
        return x['option 2']

def right_2(x):
    if x['model 1'] == compare_b:
        return x['option 1']
    else:
        return x['option 2']

display((new_df.apply(right_1, axis=1) == df[compare_a]).value_counts())
display((new_df.apply(right_2, axis=1) == df[compare_b]).value_counts())

True    110
Name: count, dtype: int64

True    110
Name: count, dtype: int64

In [16]:
file_name

'compare_csv/arc_ai2_chall_train_91_200_for_comparison.csv'

In [14]:
new_df.head()

,original,model 1,model 2,option 1,option 2
0,<question>Janet’s ducks lay 16 eggs per day. S...,claude_v2_refine,gemini_pro_think_v2,<question>הברווזים של יעל מטילים 16 ביצים ביום...,<question>הברווזים של יעל מטילים 16 ביצים ביום...
1,<question>A robe takes 2 bolts of blue fiber a...,claude_v2_refine,gemini_pro_think_v2,<question>לתפירת חלוק נדרשים 2 גלילי אריג כחול...,<question>להכנת חלוק צריך 2 גלילי בד כחול וחצי...
2,<question>Josh decides to try flipping a house...,gemini_pro_think_v2,claude_v2_refine,<question>יוסי מחליט לנסות לעשות 'פליפ' לבית. ...,"<question>יואב מחליט לנסות לעשות ""פליפ"" לדירה...."
3,<question>James decides to run 3 sprints 3 tim...,claude_v2_refine,gemini_pro_think_v2,<question>יוסי מחליט לרוץ 3 ספרינטים 3 פעמים ב...,<question>יאיר החליט לרוץ 3 ספרינטים 3 פעמים ב...
4,"<question>Every day, Wendi feeds each of her c...",gemini_pro_think_v2,claude_v2_refine,"<question>כל יום, ורד מאכילה כל אחת מהתרנגולות...","<question>בכל יום, נועה מאכילה כל אחת מהתרנגול..."


In [17]:
save_name = 'manual_compare/arc_ai2_comparison_91_200'

new_df.to_csv(save_name + '_FULL.csv', index=False)

new_df[['original', 'option 1', 'option 2']].to_excel(save_name + '.xlsx')

In [18]:
write_s = ''
for i in range(new_df.shape[0]):
    write_s += f"""Example {i}:
English:
{new_df.iloc[i]['original']}
    
Option 1:
{new_df.iloc[i]['option 1']}
    
Option 2:
{new_df.iloc[i]['option 2']}

Rating:

    
Gold:
    
""" + '-' * 50 + '\n\n'

In [19]:
with open(save_name + '.txt', 'w') as f:
    f.write(write_s)

# Compare Claude few-shots

In [27]:
[k for k in use_keys if 'claude' in k]

['claude_v1',
 'claude_v1_few_options',
 'claude_v1_refine',
 'claude_v2',
 'claude_v2_refine',
 'claude_v3',
 'claude_v4',
 'claude_v5',
 'claude_v6',
 'claude_v6_system_prompt']

In [28]:
# old = 'claude_v4'

one_v = 'claude_v1_refine'
two_v = 'claude_v2_refine'

for i in range(50):
    row = df.iloc[i]
    if row[one_v] == row[two_v]:
        continue
    else:
        print(i, '\n')
        # old_parsed = parse_query(row[old])
        one_parsed = parse_query(row[one_v])
        two_parsed = parse_query(row[two_v])
        for key in one_parsed:
            if one_parsed[key] != two_parsed[key]:
                print(f'{key}:\n\t{one_parsed[key]}\n\t{two_parsed[key]}\n')
                # print(f'{key}:\n\t{one_parsed[key]}\n\t{two_parsed[key]}\n{old}:\n{old_parsed[key]}\n')
            #     continue
            # else:
            #     print(f'{key}:\n\t{one_parsed[key]}\n\t{two_parsed[key]}\n')
        print('\n---------------------------\n')

0 

option 4:
	גבעות סלע יסוד שהתחספסו ממעבר קרחון
	גבעות סלע יסוד שהתחספסו עקב מעבר קרחון


---------------------------

1 

option 1:
	הרקמות באורגניזם חד-תאי דומות לתאים באורגניזם רב-תאי.
	רקמות באורגניזם חד-תאי דומות לתאים באורגניזם רב-תאי.

option 3:
	האברונים באורגניזם חד-תאי דומים לאיברים באורגניזם רב-תאי.
	אברונים באורגניזם חד-תאי דומים לאיברים באורגניזם רב-תאי.


---------------------------

2 

question:
	כחלק מניסוי, אסטרונאוט לוקח משקל לירח ושוקל את עצמו. המשקל מראה 31 פאונד. אם לאסטרונאוט יש מסה של כ-84 קילוגרם, מהם המשקל והמסה המקורבים של האסטרונאוט כאשר הוא עומד על כדור הארץ?
	כחלק מניסוי, אסטרונאוט לוקח משקל לירח ושוקל את עצמו. המשקל מראה 31 פאונד. אם מסת האסטרונאוט היא כ-84 קילוגרם, מהם המשקל והמסה המשוערים של האסטרונאוט כאשר הוא עומד על כדור הארץ?


---------------------------

4 

question:
	צבי ים בוקעים הם בדרך כלל בעלי צבע כהה. לעיתים, בוקע צב ים שצבעו כמעט לבן. בזמן זחילה מהקן בחוף אל האוקיינוס, צב ים בהיר זה עלול להיות בסיכון לכוויות שמש. הצבע הבהיר של הצבים יגר

In [ ]:
for i in range(50):
    print(i)
    print(text_df.loc[i, 'claude_v1_refine text'])
    print('\n- - - - - - - - - - - - - - - - - - - - - - - - - - \n')

# Compare Gemini few-shots

In [ ]:
print(df['original'].iloc[20])

In [29]:
[k for k in use_keys if 'gemini' in k]

['gemini_pro_v1',
 'gemini_pro_v1_think',
 'gemini_pro_v2',
 'gemini_pro_v4_think',
 'gemini_v1',
 'gemini_v1_few_options',
 'gemini_v1_think',
 'gemini_v2',
 'gemini_v2_think',
 'gemini_v3',
 'gemini_v3_think']

In [30]:
# old = 'claude_v4'

one_v = 'gemini_pro_v1_think'
two_v = 'gemini_pro_v4_think'

for i in range(50):
    row = df.iloc[i]
    if row[one_v] == row[two_v]:
        continue
    else:
        print(i, '\n')
        # old_parsed = parse_query(row[old])
        one_parsed = parse_query(row[one_v])
        two_parsed = parse_query(row[two_v])
        for key in one_parsed:
            if one_parsed[key] != two_parsed[key]:
                print(f'{key}:\n\t{one_parsed[key]}\n\t{two_parsed[key]}\n')
                # print(f'{key}:\n\t{one_parsed[key]}\n\t{two_parsed[key]}\n{old}:\n{old_parsed[key]}\n')
        print('\n---------------------------\n')

0 

option 2:
	ערמות סלעים שהושקעו על ידי קרחון נמס
	ערמות של סלעים שהושקעו על ידי קרחון נמס

option 3:
	חריצים שנוצרו במשטח גרניט על ידי קרחון
	חריצים שנוצרו על משטח גרניט על ידי קרחון


---------------------------

1 

question:
	איזה היגד משווה בצורה הטובה ביותר בין יצורים חד-תאיים לרב-תאיים?
	איזה היגד משווה בצורה הטובה ביותר בין אורגניזמים חד-תאיים לרב-תאיים?

option 1:
	רקמות ביצור חד-תאי הן כמו התאים ביצור רב-תאי.
	רקמות באורגניזם חד-תאי הן כמו התאים באורגניזם רב-תאי.

option 2:
	גרעין התא ביצור חד-תאי הוא כמו העור של יצור רב-תאי.
	הגרעין באורגניזם חד-תאי הוא כמו העור של אורגניזם רב-תאי.

option 3:
	אברונים ביצור חד-תאי הם כמו האיברים ביצור רב-תאי.
	אברונים באורגניזם חד-תאי הם כמו האיברים באורגניזם רב-תאי.

option 4:
	הציטופלזמה ביצור חד-תאי היא כמו מערכת העצבים ביצור רב-תאי.
	הציטופלזמה באורגניזם חד-תאי היא כמו מערכת העצבים באורגניזם רב-תאי.


---------------------------

4 

question:
	צבי ים שבקעו מהביצה הם בדרך כלל כהים. מדי פעם, בוקע צב ים שכמעט לבן. בזמן הזחילה מהקן על החו

In [32]:
text_df.head(2)

,original,claude_v1_few_options text,claude_v1_refine text,gemini_pro_v4_think text,claude_v2_refine text
0,<question>Which land form is the result of the...,I'll translate the question and answer options...,First translation:\n<question>איזו צורת קרקע ה...,**The Translation Process: A Geologist's Persp...,First translation:\n<question>איזו צורת קרקע ה...
1,<question>Which statement best compares single...,I'll translate the English question and answer...,First translation:\n<question>איזה היגד משווה ...,**The Translation Process: A Biologist's Persp...,First translation:\n<question>איזה היגד משווה ...


In [33]:
for i in range(50):
    print(i)
    print(text_df.loc[i, 'gemini_pro_v4_think text'])
    print('\n- - - - - - - - - - - - - - - - - - - - - - - - - - \n')

0
**The Translation Process: A Geologist's Perspective**

Okay, so I've got this English science question about glaciers and landforms, and the task is to translate it and its multiple-choice answers into Hebrew. My first thought is always to consider the nuances of scientific terminology.

First, the question itself: "Which land form is the result of the constructive force of a glacier?" "Land form" immediately makes me think of the formal Hebrew term, *תצורת נוף* (t'zurat nof), although *צורת נוף* (tzurat nof) is also acceptable. Given the slightly formal nature of the question, I opted for *תצורת נוף*.  "Constructive force" is straightforward – *הכוח הבונה* (ha-ko'ach ha-boneh) – building up, as opposed to erosion. So, the question becomes: "איזו תצורת נוף היא תוצאה של הכוח הבונה של קרחון?" (Which land form is the result of the constructive force of a glacier?). Sounds good to me.

Now, for the options.  Option 1, "valleys carved by a moving glacier," jumps out as erosional.  I tran